In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()


MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


212990

In [3]:
users_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allUsers/*.gz")
print users_lines
users_lines.count()

MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2


100425

In [4]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime
import time


In [5]:
def parseUser(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "CreationDate" in root.attrib:
                return(root.attrib["Id"], root.attrib["CreationDate"])
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            

In [6]:
users = users_lines.map(parseUser).filter(lambda x: x!= 'Empty')
print users.count()
print users.take(20)

50320
[('70185', '2015-03-02T18:42:20.510'), ('70186', '2015-03-02T19:04:13.380'), ('70187', '2015-03-02T19:40:16.420'), ('70188', '2015-03-02T19:46:45.400'), ('70189', '2015-03-02T19:56:37.233'), ('70190', '2015-03-02T19:59:18.320'), ('70191', '2015-03-02T20:08:27.600'), ('70192', '2015-03-02T20:10:19.623'), ('70193', '2015-03-02T20:41:46.567'), ('70194', '2015-03-02T20:46:08.073'), ('70195', '2015-03-02T20:52:08.043'), ('70196', '2015-03-02T20:57:18.773'), ('70197', '2015-03-02T21:08:21.617'), ('70198', '2015-03-02T21:37:38.363'), ('70199', '2015-03-02T21:38:27.963'), ('70200', '2015-03-02T22:35:10.230'), ('70201', '2015-03-02T22:39:31.877'), ('70202', '2015-03-02T23:33:39.623'), ('70203', '2015-03-02T23:41:20.063'), ('70204', '2015-03-02T23:45:15.587')]


In [7]:
def parsePost(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    return (root.attrib["OwnerUserId"], root.attrib["CreationDate"])
                except:
                    pass
                    return ("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [8]:
posts = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')
print posts
print posts.take(20)
posts.count()


PythonRDD[8] at RDD at PythonRDD.scala:43
[('16609', '2014-06-04T13:18:18.120'), ('30623', '2014-06-04T13:31:24.160'), ('45266', '2014-06-04T13:43:15.913'), ('46059', '2014-06-04T13:57:07.657'), ('46703', '2014-06-04T14:14:48.753'), ('7828', '2014-06-04T14:19:20.430'), ('29068', '2014-06-04T14:25:45.873'), ('15997', '2014-06-04T14:28:42.040'), ('45277', '2014-06-04T14:30:15.493'), ('7916', '2014-06-04T14:35:08.560'), ('29068', '2014-06-04T14:35:51.100'), ('39953', '2014-06-04T14:41:11.140'), ('46705', '2014-06-04T14:48:17.310'), ('46708', '2014-06-04T15:00:58.427'), ('21160', '2014-06-04T15:07:20.207'), ('15997', '2014-06-04T15:07:48.407'), ('29068', '2014-06-04T15:07:55.817'), ('45268', '2014-06-04T15:13:20.170'), ('8013', '2014-06-04T15:14:33.713'), ('7828', '2014-06-04T15:24:10.153')]


107152

In [25]:
def parseQuestion(line):
    if '<row'in line:
        try:
            root = ET.fromstring(line.encode('utf8').strip())
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" in root.attrib:
                p_type = root.attrib["PostTypeId"]
                owner = root.attrib["OwnerUserId"]
                if p_type == '1':
                    score = float(root.attrib["Score"])
                    views = float(root.attrib["ViewCount"])
                    answers = float(root.attrib["AnswerCount"])
                    create_date = root.attrib["CreationDate"] 
                    favs = 0.
                    try:
                        favs = float(root.attrib["FavoriteCount"])
                    except:
                        pass

                    return(owner, [create_date, score, views, answers, favs])
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [26]:
questions = posts_lines.map(parseQuestion).filter(lambda x: x!= "Empty")\
            .reduceByKey(min).map(lambda x: (x[0],x[1][1:]))
print questions.count()
print questions.take(20)

23133
[('23994', [-1.0, 33.0, 1.0, 0.0]), ('61518', [4.0, 173.0, 1.0, 1.0]), ('35549', [1.0, 390.0, 1.0, 0.0]), ('36313', [3.0, 73.0, 0.0, 0.0]), ('69951', [0.0, 15.0, 0.0, 0.0]), ('11549', [2.0, 304.0, 1.0, 1.0]), ('60913', [1.0, 12.0, 0.0, 0.0]), ('17525', [3.0, 65.0, 2.0, 0.0]), ('11384', [1.0, 264.0, 0.0, 0.0]), ('43270', [0.0, 69.0, 1.0, 0.0]), ('10897', [1.0, 229.0, 1.0, 2.0]), ('48893', [3.0, 390.0, 1.0, 1.0]), ('3993', [8.0, 2850.0, 4.0, 1.0]), ('61176', [1.0, 81.0, 0.0, 0.0]), ('56583', [0.0, 18.0, 1.0, 0.0]), ('4686', [1.0, 70.0, 0.0, 0.0]), ('25065', [4.0, 89.0, 1.0, 3.0]), ('56236', [0.0, 36.0, 1.0, 0.0]), ('12703', [2.0, 238.0, 0.0, 1.0]), ('61195', [0.0, 15.0, 0.0, 0.0])]


In [27]:
from datetime import datetime
import time

def dateConversion (date1, date2):  
    date1_obj = datetime.strptime(date1, "%Y-%m-%dT%H:%M:%S.%f")
    date2_obj = datetime.strptime(date2, "%Y-%m-%dT%H:%M:%S.%f")
    diff = date2_obj - date1_obj
    diff_sec = diff.total_seconds()
    if diff_sec > 8640000 and diff_sec < 12960000:
        return 1
    else:
        return 0
  

In [53]:
def transform(x):
    if x[1]>=1:
        vet =1
    else:
        vet = 0
    return (x[0],vet)

In [54]:
joint = users.join(posts).map(lambda x: (x[0], dateConversion(x[1][0], x[1][1])))\
        .reduceByKey(lambda x,y: x+y).map(transform)
print joint.count()
print joint.take(40)

26847
[('23991', 0), ('55619', 0), ('55344', 0), ('35549', 0), ('22465', 0), ('69953', 0), ('13732', 1), ('11549', 0), ('28999', 0), ('4028', 0), ('11386', 0), ('55298', 0), ('7515', 1), ('48899', 0), ('55766', 0), ('22269', 0), ('61205', 0), ('16674', 0), ('25068', 0), ('52265', 0), ('70001', 0), ('6154', 0), ('60355', 0), ('4027', 0), ('46135', 0), ('298', 0), ('41847', 0), ('64700', 0), ('64877', 0), ('21100', 0), ('60135', 0), ('17651', 0), ('57297', 0), ('46488', 0), ('43988', 0), ('52399', 0), ('36207', 0), ('41838', 1), ('57168', 0), ('56830', 0)]


In [55]:
veterans = joint.join(questions).map(lambda x: (x[1][0], x[1][1]+[1]))\
            .reduceByKey(lambda x,y: np.add(x,y))\
            .map(lambda x: (x[0], [x[1][0]/x[1][4], x[1][1]/x[1][4], x[1][2]/x[1][4], x[1][3]/x[1][4]]))
print veterans.count()
print veterans.take(40)

2
[(0, [2.1008130845513935, 553.51919913521647, 0.97071955632842977, 0.57588005827889266]), (1, [3.5434543454345433, 926.39823982398241, 1.2981298129812981, 1.3008800880088009])]
